In [ ]:
!pip install torch transformers accelerate bitsandbytes peft sentence-transformers faiss-cpu langchain langchain-community langchain-huggingface pdfplumber huggingface_hub trl

In [ ]:
from huggingface_hub import login
login(new_session=False)

In [ ]:
!wandb login

In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
)
from transformers.trainer_utils import get_last_checkpoint
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer
import os


VRAM_PROFILE = "gpu"
MAX_TRAINING_STEPS = 170
CHECKPOINT_SAVE_STEPS = 50
CHECKPOINT_SAVE_LIMIT = 2
MODEL_NAME = "Qwen/Qwen2-1.5B-Instruct"
DATASET_FILE = "legal_dataset.json"
NEW_MODEL_NAME = "qwen2-1.5b-russian-law-expert"
OUTPUT_DIR = "./results"

batch_size = 4
grad_accumulation_steps = 2
lora_rank = 8


print(f" Загрузка базовой модели: {MODEL_NAME}")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
)
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print(f"Загрузка датасета из файла: {DATASET_FILE}")
dataset = load_dataset("json", data_files=DATASET_FILE, split="train")

def format_dataset_function(example):
    return {"text": f"Инструкция: {example['instruction']}\nОтвет: {example['output']}"}

dataset = dataset.map(format_dataset_function)
print(" Датасет отформатирован и содержит колонку 'text'.")


# --- 5. Конфигурация LoRA ---
lora_config = LoraConfig(
    r=lora_rank,
    lora_alpha=lora_rank * 2,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj"]
)
model = get_peft_model(model, lora_config)

# --- 6. Настройки обучения ---
training_arguments = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=grad_accumulation_steps,
    optim="adamw_torch",
    save_strategy="steps",
    save_steps=CHECKPOINT_SAVE_STEPS,
    save_total_limit=CHECKPOINT_SAVE_LIMIT,
    max_steps=MAX_TRAINING_STEPS,
    logging_steps=10,
    learning_rate=2e-4,
    fp16=False,
    bf16=True,
    group_by_length=True,
    lr_scheduler_type="constant",
)

# --- 7. Создание тренера ---
print(" Создание тренера SFTTrainer (минимальная конфигурация)...")
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=lora_config,
    args=training_arguments,
)

print("Начинаем fine-tuning ...")

last_checkpoint = get_last_checkpoint(OUTPUT_DIR)
if last_checkpoint:
    print(f"Возобновление обучения с чекпоинта: {last_checkpoint}")
else:
    print("Начинаем обучение с нуля.")

trainer.train(resume_from_checkpoint=last_checkpoint)


print(f" Обучение завершено. Сохраняем финальный адаптер в папку ./{NEW_MODEL_NAME}")
trainer.model.save_pretrained(NEW_MODEL_NAME)
tokenizer.save_pretrained(NEW_MODEL_NAME)


In [ ]:
import os
import gc
import torch
import pdfplumber
import gradio as gr
from typing import List
from peft import PeftModel
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig


BASE_MODEL_NAME = "Qwen/Qwen2-1.5B-Instruct"
LORA_ADAPTER_PATH = "./qwen2-1.5b-russian-law-expert"
PDF_FILES_DIR = "./data"
FAISS_INDEX_PATH = "./faiss_index"
EMBEDDING_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"

model = None
tokenizer = None
embeddings = None
retriever = None

def load_finetuned_model_memory_optimized(base_model_path, peft_adapter_path):
    print("Загрузка базовой модели с 4-битной квантизацией...")

    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        base_model_path,
        dtype=torch.bfloat16,
        quantization_config=quantization_config,
        device_map="auto"
    )

    print(f"Применение LoRA адаптера из: {peft_adapter_path}")
    model = PeftModel.from_pretrained(model, peft_adapter_path)

    print("Модель и адаптер загружены без слияния для экономии памяти.")

    tokenizer = AutoTokenizer.from_pretrained(base_model_path)
    tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer


def fast_load_pdf(path: str) -> List[Document]:
    local_docs = []
    try:
        with pdfplumber.open(path) as pdf:
            for i, page in enumerate(pdf.pages):
                text = page.extract_text()
                if text and text.strip():
                    local_docs.append(Document(
                        page_content=text,
                        metadata={"source": os.path.basename(path), "page": i + 1}
                    ))
    except Exception as e:
        print(f"Ошибка при чтении файла {path}: {e}")
    return local_docs

def create_or_load_faiss_index_batched(pdf_dir, index_path, embeddings_model):
    if os.path.exists(index_path):
        print(f"Загрузка существующей FAISS базы из {index_path}...")
        return FAISS.load_local(index_path, embeddings_model, allow_dangerous_deserialization=True)

    print(f"Создание новой FAISS базы из PDF в папке {pdf_dir} (пофайловая обработка)...")
    pdf_files = [os.path.join(pdf_dir, f) for f in os.listdir(pdf_dir) if f.endswith(".pdf")]
    if not pdf_files:
        os.makedirs(pdf_dir, exist_ok=True)
        return None


    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)

    print(f"  -> Обработка файла 1/{len(pdf_files)}: {os.path.basename(pdf_files[0])}")
    initial_docs = fast_load_pdf(pdf_files[0])
    initial_chunks = splitter.split_documents(initial_docs)

    if not initial_chunks:
         raise ValueError(f"Первый PDF файл {pdf_files[0]} пуст или не удалось извлечь текст.")

    print(f"  -> Создание начального индекса...")
    vectorstore = FAISS.from_documents(initial_chunks, embeddings_model)

    if len(pdf_files) > 1:
        for i, pdf_file in enumerate(pdf_files[1:], start=2):
            print(f"  -> Обработка файла {i}/{len(pdf_files)}: {os.path.basename(pdf_file)}")
            docs = fast_load_pdf(pdf_file)
            if not docs:
                print(f"    -- Пропущен пустой файл.")
                continue
            chunks = splitter.split_documents(docs)
            print(f"    -> Добавление {len(chunks)} чанков в индекс...")
            vectorstore.add_documents(chunks)
            del docs, chunks
            gc.collect()

    print(f"Сохранение финального индекса в {index_path}...")
    vectorstore.save_local(index_path)
    print(f"База создана и сохранена.")
    return vectorstore


def get_answer(query: str, current_model, current_tokenizer, current_retriever):
    if not current_retriever:
        return "Ошибка: База знаний не загружена. Пожалуйста, убедитесь, что PDF файлы находятся в папке './data' и перезапустите приложение.", []

    print(f"\nПоиск релевантных документов для запроса: '{query}'")
    context_docs = current_retriever.invoke(query)
    context_text = "\n\n".join([f"Источник: {doc.metadata['source']}, стр. {doc.metadata['page']}\n---\n{doc.page_content}" for doc in context_docs])

    prompt = f"""Инструкция: Ты — высококвалифицированный юрист-консультант. Используй предоставленный ниже контекст из юридических документов, чтобы дать точный и ясный ответ на вопрос пользователя. Ссылайся на источники, если это уместно.

**Контекст из документов:**
{context_text}

**Вопрос пользователя:**
{query}

Ответ:"""

    print("Модель генерирует ответ...")
    messages = [{"role": "user", "content": prompt}]
    input_ids = current_tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(current_model.device)

    outputs = current_model.generate(
        input_ids,
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.2,
        top_p=0.9,
    )
    response = current_tokenizer.decode(outputs[0][input_ids.shape[1]:], skip_special_tokens=True)

    sources_formatted = "\n".join([f"- {doc.metadata['source']}, страница {doc.metadata['page']}" for doc in context_docs])
    return response, sources_formatted


def gradio_interface(question: str):
    global model, tokenizer, retriever
    if not model or not tokenizer or not retriever:
        return "Система не инициализирована. Пожалуйста, подождите или перезапустите.", ""

    answer, sources = get_answer(question, model, tokenizer, retriever)
    return answer, sources

def initialize_system():
    global model, tokenizer, embeddings, retriever

    if not os.path.isdir(LORA_ADAPTER_PATH):
        print(
            f"Папка с адаптером не найдена по пути: '{LORA_ADAPTER_PATH}'. "
            "Проверьте опечатки и убедитесь, что вы загрузили папку в текущую директорию."
        )
        return False, f"Ошибка: Папка с LoRA адаптером не найдена: '{LORA_ADAPTER_PATH}'."

    try:
        model, tokenizer = load_finetuned_model_memory_optimized(BASE_MODEL_NAME, LORA_ADAPTER_PATH)

        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)
        vectorstore = create_or_load_faiss_index_batched(PDF_FILES_DIR, FAISS_INDEX_PATH, embeddings)

        if vectorstore is None:
            return False, f"Ошибка: В папке '{PDF_FILES_DIR}' не найдено PDF файлов. Пожалуйста, добавьте PDF документы для создания базы знаний."

        retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

        print("\n\n--- RAG-помощник по законам РФ готов к работе! ---")
        return True, "Система успешно инициализирована и готова к работе."

    except Exception as e:
        print(f"Произошла ошибка при инициализации системы: {e}")
        return False, f"Ошибка инициализации: {e}"


if __name__ == "__main__":
    init_success, init_message = initialize_system()

    if not init_success:
        with gr.Blocks() as demo:
            gr.Markdown("# RAG-помощник по законам РФ (Ошибка инициализации)")
            gr.Textbox(value=init_message, label="Статус системы", interactive=False)
            gr.Markdown("Пожалуйста, устраните указанные проблемы (например, добавьте PDF-файлы в папку './data') и перезапустите приложение.")
        demo.launch(share=True)
    else:
        with gr.Blocks(theme=gr.themes.Soft()) as demo:
            gr.Markdown("# RAG-помощник по законам РФ")
            gr.Markdown("Задайте вопрос по российскому законодательству, и я постараюсь ответить, используя предоставленные PDF-документы.")

            with gr.Row():
                with gr.Column(scale=2):
                    question_input = gr.Textbox(label="Ваш вопрос", placeholder="Например: 'Какие права имеет потребитель при возврате товара?'", lines=5)
                    submit_btn = gr.Button("Получить ответ")
                with gr.Column(scale=3):
                    answer_output = gr.Textbox(label="Ответ", interactive=False, lines=10)
                    sources_output = gr.Textbox(label="Использованные источники", interactive=False, lines=5)

            submit_btn.click(
                fn=gradio_interface,
                inputs=question_input,
                outputs=[answer_output, sources_output]
            )

            gr.Examples(
                examples=[
                    "Какие основные положения содержит закон о защите прав потребителей?",
                    "Каков порядок регистрации юридического лица?",
                    "Какие существуют виды уголовных наказаний в РФ?"
                ],
                inputs=question_input
            )

            gr.Textbox(value=init_message, label="Статус загрузки системы", interactive=False)


        demo.launch(share=True)